In [3]:
import numpy as np
import deepxde as dde
from deepxde.backend import tensorflow_compat_v1
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [4]:
x_min, x_max = -np.pi, np.pi
t_min, t_max = 0.0, 2*np.pi

x = np.linspace(x_min, x_max, 256)
t = np.linspace(t_min, t_max, 201)
X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

space_domain = dde.geometry.Interval(x_min, x_max)
time_domain = dde.geometry.TimeDomain(t_min, t_max)
geomtime = dde.geometry.GeometryXTime(space_domain, time_domain)

In [ ]:
def pde(x, y):
    """
    INPUTS:
        x: x[:,0] is x-coordinate
           x[:,1] is t-coordinate
        y: Network output, in this case:
            y[:,0] is u(x,t) the real part
            y[:,1] is v(x,t) the imaginary part
    OUTPUT:
        The pde in standard form i.e. something that must be zero
    """
    u = y[:, 0:1]
    v = y[:, 1:2]

    # In 'jacobian', i is the output component and j is the input component
    u_t = dde.grad.jacobian(y, x, i=0, j=1)
    v_t = dde.grad.jacobian(y, x, i=1, j=1)
    u_x = dde.grad.jacobian(y, x, i=0, j=0)
    v_x = dde.grad.jacobian(y, x, i=1, j=0)

    # In 'hessian', i and j are both input components. (The Hessian could be in principle something like d^2y/dxdt, d^2y/d^2x etc)
    # The output component is selected by "component"
    u_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    v_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)

    f_u = v_t - 0.5 * (u_xx - x[:, 0:1]**2*u)
    f_v = u_t + 0.5 * (v_xx - x[:, 0:1]**2*v)

    return [f_u, f_v]

In [ ]:
def psi_init_u(x):
    u_0 = (1.0/np.pi)**0.25 * np.exp(-0.5*x[:, 0:1]**2)
    u_1 = (1.0/np.pi)**0.25 * np.exp(-0.5*x[:, 0:1]**2) * x[:, 0:1] * np.sqrt(1)
    return (1.0/np.sqrt(2.0)) * (u_0 + u_1)

def psi_init_v(x):
    return 0

def boundary(x, on_boundary):
    return on_boundary

bc_u_0 = dde.icbc.boundary_conditions.DirichletBC(geomtime, lambda x: 0, boundary, component=0)
bc_v_0 = dde.icbc.boundary_conditions.DirichletBC(geomtime, lambda x: 0, boundary, component=1)
